In [1]:
from waylay import WaylayClient, RestResponseError
waylay_client = WaylayClient.from_profile('demo')

# Waylay Python SDK - Making REST calls

The Waylay Python SDK exposes a number of _REST Services_, each a collection of _REST Resources_ that have _action methods_.

For an overview, see 
> `[Enterprise]` https://docs.waylay.io/#/api/sdk/python

In most cases these _action methods_ directly correspond to an underlying REST endpoint of the waylay platform.

In general, the Python SDK takes care of
* authentication
* the endpoint url to call, and HTTP method to use
* converting JSON responses to Python objects, extracting the relevant data
* converting Python request body objects to JSON
* handling errors 

Check out the documentation sites ([enterprise](https://docs.waylay.io/#/api/sdk/python)) on

* how requests and response are exactly represented as json
* which additional parameters can be provided
* what the preconditions and effects are of your call

### request arguments
These are the general rules for making requests, corresponding to the REST documentation of the underlying REST action:
* url **path parameters** are passed as **positional arguments**:
  ###### example

  > `waylay_client.queries.query.get('151CF-temperature')` 

  will use 151CF-temperature to construct a request to fetch this query definition

  > `GET https://api.waylay.io/queries/v1/query/151CF-temperature` 

* **query parameters** belong in a named **`params` argument**:
  ###### example
  > `waylay_client.queries.query.execute('151CF-temperature', params={'from':'2021-03-01T10:00:00+00:00'})`
  
  will bind the `from` parameter

* **request objects** are passed into a **`body` argument**:
  ###### example
  > `waylay_client.queries.query.execute(body={'resource':'RDJ_89839','metric':'revolutions'})`}|

### response handling
These are the general rules on how the SDK treats results of REST calls:
* Only successfull responses (HTTP status `2XX`) return a result, other responses raise an exception.
* The SDK extracts the most relevant part of the (json) response body. You can use the `select_path` argument to override this behaviour.
* When the REST call produces **timeseries data**, the SDK will return a **pandas DataFrame**.
* By providing the  `raw=True` argument, you instruct the SDK to skip all error and response handling, and **just return the HTTP response**.

### error handling

The SDK uses the following exception hierarchy to notify problems. These exception classes belong to the `waylay.exception` module. They all descend from a `WaylayError` base class.

| exception class | raised in case of |
| --------------- | ----------- |
| `AuthError` | Waylay authentication errors. |
| `ConfigError` |  Waylay client configuration errors. |
| `RequestError` | Errors in tools and utilities that are not directly related to a REST call. |
| `RestRequestError` | Failure to prepare a REST call. |
| `RestResponseError` | Wraps the result of a failed REST call. |
| `RestResponseParseError` | Failure to parse the result of a succesfull REST call. |

Errors of type `RestResponse` have the following attributes that you can use to handle problems:

* `response` contains the full HTTP Response object of the REST call, which lets you inspect the status code (`response.status_code`) , response body (`response.body`) and other attributes such as _headers_.
* `message` will give you the most relevant error message




### http response information

Most _action methods_ support a `raw=True` parameter. This will prevent exception handling and parsing of the REST call by the SDK. The unparsed result and http response information is returned in a _Response_ object with attributes
 * `body` : the result data (json and csv data is parsed to python data structures)
 * standard http information such as `url`, `method`, `headers`, `status_code`, `client_response`

In [2]:
waylay_client.queries.query.list()

['151CF-temperature',
 '151CF-temperature-demo',
 '151CF-temperature-wrong',
 ' battery-life-testset',
 'battery-life-testset',
 'battery-life-testset2',
 'battery-life-testset-sdk0.3',
 'BearingQuery',
 'BearingQuery_EngineA',
 'BearingQuery_EngineX']

In [3]:
http_response = waylay_client.queries.query.get('battery-life-testset', raw=True)
{
    'url':http_response.url,
    'method':http_response.method,
    'content-type':http_response.headers['content-type'],
    'status_code':http_response.status_code,
    'name':http_response.body['name'],
}


{'url': 'https://api.waylay.io/queries/v1/query/battery-life-testset',
 'method': 'GET',
 'content-type': 'application/json',
 'status_code': 200,
 'name': 'battery-life-testset'}

### access http response information from an error
When an request is unsuccessfull, the client will raise an exception. 

These exceptions (from the `waylay.exceptions` module) are either (instances of subclasses of)
* a `RestRequestError` that indicates a problem before sending an api call to waylay (e.g. when input argument conversion fails)
* a `RestResponseError` that reports a problem from or after the api call to waylay. This exception gives you access to the underlying response (`response` attribute)
  * a `RestResponseParseError` error indicates a problem in processing a succesfull response from the waylay platform. All other `RestResponseError` will come from errors reported by the waylay platform itself (http status code above the `200` range).
  
Other errors can occur (such as standard python `ValueError`,`TypeError` or `AttributeError`) but these will normally indicate a programming error. Networking failures will normally result in a `ClientConnectionError`

In [4]:
# try to get the representation of a `query` entity that does not exist. 
# this will result in a `404 NOT FOUND` error 
try:
   waylay_client.queries.query.get('where are you???')
except RestResponseError as exc:
   print(exc.message)
   print(exc.response.status_code)
   print(exc.response.url)
    


operation=not_found_error
404
https://api.waylay.io/queries/v1/query/where%20are%20you?%3F%3F=


## accessing the complete response
The SDK extracts the most relevant part of the (json) response body. You can use the `select_path` argument to override this behaviour

In [5]:
# the `query` entity in the REST response is given as default response on the SDK action
waylay_client.queries.query.get('battery-life-testset')

{'data': [{'metric': 'cycle_number', 'resource': 'battery-life-testset'},
  {'metric': 'policy', 'resource': 'battery-life-testset'},
  {'metric': 'IR', 'resource': 'battery-life-testset'},
  {'metric': 'QD', 'resource': 'battery-life-testset'},
  {'metric': 'Discharge_time', 'resource': 'battery-life-testset'},
  {'metric': 'Qdlin_mean', 'resource': 'battery-life-testset'},
  {'metric': 'Qdlin_first', 'resource': 'battery-life-testset'},
  {'metric': 'Qdlin_last', 'resource': 'battery-life-testset'},
  {'metric': 'Qdlin_median', 'resource': 'battery-life-testset'},
  {'metric': 'Qdlin_25p', 'resource': 'battery-life-testset'},
  {'metric': 'Qdlin_75p', 'resource': 'battery-life-testset'},
  {'metric': 'Qdlin_0.1', 'resource': 'battery-life-testset'},
  {'metric': 'Qdlin_0.9', 'resource': 'battery-life-testset'},
  {'metric': 'Vdlin_mean', 'resource': 'battery-life-testset'},
  {'metric': 'Vdlin_first', 'resource': 'battery-life-testset'},
  {'metric': 'Vdlin_last', 'resource': 'batter

In [6]:
waylay_client.queries.query.get('battery-life-testset', select_path=False)

{'_links': {'self': {'href': 'https://api.waylay.io/queries/v1/query/battery-life-testset'},
  'execute': {'href': 'https://api.waylay.io/queries/v1/data/battery-life-testset'}},
 'attrs': {'created': '2021-11-08T12:39:27.625655+00:00',
  'created_by': 'users/f16ccae4-8f0f-4f39-912b-5c2cb13ff6b6',
  'modified': '2021-11-08T12:39:27.625655+00:00',
  'modified_by': 'users/f16ccae4-8f0f-4f39-912b-5c2cb13ff6b6',
  'path': '/tsa/query/battery-life-testset'},
 'name': 'battery-life-testset',
 'meta': {},
 'query': {'data': [{'metric': 'cycle_number',
    'resource': 'battery-life-testset'},
   {'metric': 'policy', 'resource': 'battery-life-testset'},
   {'metric': 'IR', 'resource': 'battery-life-testset'},
   {'metric': 'QD', 'resource': 'battery-life-testset'},
   {'metric': 'Discharge_time', 'resource': 'battery-life-testset'},
   {'metric': 'Qdlin_mean', 'resource': 'battery-life-testset'},
   {'metric': 'Qdlin_first', 'resource': 'battery-life-testset'},
   {'metric': 'Qdlin_last', 'reso

In [7]:
waylay_client.queries.query.get('battery-life-testset', select_path=['attrs','modified'])

'2021-11-08T12:39:27.625655+00:00'

## customise or replace dataframe conversions
The methods of the `queries.query` that return timeseries data, will normally return their data as a pandas `DataFrame`. These have immediate display support in jupyter notebooks:

In [8]:
waylay_client=WaylayClient.from_profile('staging')
waylay_client.queries.query.execute(
    'weatherHistoryDaily'
)

resource                         06e267b9-8714-4c58-ac53-df6a291b59dc  \
metric                                                    temperature   
timestamp                                                               
2023-10-12 03:21:43.794000+00:00                                15.11   
2023-10-13 03:21:44.306000+00:00                                17.89   
2023-10-14 03:21:43.979000+00:00                                10.50   
2023-10-15 03:21:43.698000+00:00                                 7.05   
2023-10-16 03:21:44.505000+00:00                                 4.48   

resource                                                                 
metric                           temp_feel pressure wind_speed humidity  
timestamp                                                                
2023-10-12 03:21:43.794000+00:00     15.18   1012.0       3.06     96.0  
2023-10-13 03:21:44.306000+00:00     18.22   1009.0       5.60     95.0  
2023-10-14 03:21:43.979000+00:00      9.46   1009.0       4.98     71.0  
2023-10-15 03:21:43.698000+00:00      4.45   1020.0       3.86     79.0  
2023-10-16 03:21:44.505000+00:00      3.51   1023.0       1.39     68.0

You can use the `response_constructor` parameter to replace the dataframe constructor with your own method. 

Other _action methods_ that apply conversions to raw json data structures normally support this same optional parameter.

A falsy value will let the method return just the json data payload (as python object).

In [9]:
waylay_client.queries.query.execute(
    'weatherHistoryDaily', 
    params={ 'from': '-P30D', 'freq': 'P7D', 'aggregation': 'max'},
    response_constructor=False
)

[{'columns': ['timestamp',
   {'resource': '06e267b9-8714-4c58-ac53-df6a291b59dc',
    'metric': 'temperature',
    'aggregation': 'mean'},
   {'resource': '06e267b9-8714-4c58-ac53-df6a291b59dc',
    'metric': 'temp_feel',
    'aggregation': 'mean'},
   {'resource': '06e267b9-8714-4c58-ac53-df6a291b59dc',
    'metric': 'pressure',
    'aggregation': 'mean'},
   {'resource': '06e267b9-8714-4c58-ac53-df6a291b59dc',
    'metric': 'wind_speed',
    'aggregation': 'mean'},
   {'resource': '06e267b9-8714-4c58-ac53-df6a291b59dc',
    'metric': 'humidity',
    'aggregation': 'mean'}],
  'data': [[1694649600000,
    15.110000000000001,
    14.694285714285712,
    1012.5714285714286,
    3.6271428571428572,
    77.28571428571429],
   [1695254400000,
    12.522857142857143,
    11.778571428571427,
    1012.2857142857143,
    3.8700000000000006,
    80.14285714285714],
   [1695859200000,
    14.772857142857143,
    14.378571428571428,
    1018.7142857142857,
    4.32,
    79.42857142857143],
   [1

In [10]:
# return a map with timestamps as keys, the observation values as value. 
waylay_client.queries.query.execute(
    'weatherHistoryDaily', 
    params={ 'from': '-P30D', 'freq': 'P7D', 'aggregation': 'max'},
    response_constructor=lambda d: { row[0] : row[1:] for row in d[0]['data'] }
)

{1694649600000: [15.110000000000001,
  14.694285714285712,
  1012.5714285714286,
  3.6271428571428572,
  77.28571428571429],
 1695254400000: [12.522857142857143,
  11.778571428571427,
  1012.2857142857143,
  3.8700000000000006,
  80.14285714285714],
 1695859200000: [14.772857142857143,
  14.378571428571428,
  1018.7142857142857,
  4.32,
  79.42857142857143],
 1696464000000: [13.358571428571429,
  12.884285714285715,
  1021.5714285714286,
  3.742857142857143,
  81.71428571428571],
 1697068800000: [11.006, 10.164, 1014.6, 3.778, 81.8]}

In [11]:
import numpy as np
# return timeseries data as a numpy array, transposed as an array per series 
waylay_client.queries.query.execute(
    'weatherHistoryDaily', 
    params={ 'from': '-P30D', 'freq': 'P7D', 'aggregation': 'max'},
    response_constructor=lambda d: np.transpose(d[0]['data'])
)

array([[1.69464960e+12, 1.69525440e+12, 1.69585920e+12, 1.69646400e+12,
        1.69706880e+12],
       [1.51100000e+01, 1.25228571e+01, 1.47728571e+01, 1.33585714e+01,
        1.10060000e+01],
       [1.46942857e+01, 1.17785714e+01, 1.43785714e+01, 1.28842857e+01,
        1.01640000e+01],
       [1.01257143e+03, 1.01228571e+03, 1.01871429e+03, 1.02157143e+03,
        1.01460000e+03],
       [3.62714286e+00, 3.87000000e+00, 4.32000000e+00, 3.74285714e+00,
        3.77800000e+00],
       [7.72857143e+01, 8.01428571e+01, 7.94285714e+01, 8.17142857e+01,
        8.18000000e+01]])